In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:

from dataclasses import dataclass
import asyncio 
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)

In [14]:
from agents import Agent, Runner, set_tracing_disabled,function_tool 
 

@function_tool 
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    
    resp = f"priminister  {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

@function_tool 
def find_prisedent(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime prisident of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    
    resp = f"President of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp


agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister,find_prisedent
    ],
    model=model,
    tool_use_behavior={"stop_at_tool_names": ["find_prisedent"]}
    
)


In [15]:

resp = await Runner.run(agent, "who is president of pakistan in 2021",max_turns=2)
print(resp.final_output)

 President of pakistan in 2021 is Imran Khan 
President of pakistan in 2021 is Imran Khan


In [16]:

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021",max_turns=2)
print(resp.final_output)

 priminister  pakistan in 2021 is Imran Khan 
The prime minister of Pakistan in 2021 was Imran Khan.



In [ ]:
from agents import Agent, function_tool
import json


def dynamic_instruction(context, agent):
    try:
        result = {
            "message": "Order(s) retrieved successfully.",
            "items": [
                {
                    "shipping_address": "village lakra sialkot,pakistan",
                    "tracking_id": "ASM90329313",
                    "status": "pending",
                    "expected_delivery_date": "2025-05-03",
                    "created_at": "2025-04-26",
                    "items": [
                        {
                            "product_name": "Nike Tiempo Legend 10 Elite",
                            "price": 240.0,
                            "quantity": 2,
                            "color": "Metallic Silver/Volt/Black",
                            "size": 8.5,
                            "image": "https://static.nike.com/a/images/t_default/53496442-075a-4e82-9fb4-8759b8e7285d/LEGEND+10+ELITE+AG-PRO.png",
                        }
                    ],
                }
            ],
        }
    system_prompt = f""" You are a smart and friendly order tracking assistant. 
Your job is to:
- Return only the **specific information** the user asks for (e.g., color, size, product name, image, etc.).
- Format every response using **valid HTML**, so it can be rendered safely in a Next.js frontend.
- Never fabricate or guess any order details — only respond with data that actually exists.
- If the user asks for an image, use the following format for compatibility with Next.js:

```html
<Image src="[image_url]" alt="Product image" layout="fill" objectFit="contain" />
````

---

### 🔁 Examples of Expected Behavior

**User:** What is the product name?
**Response:**

```html
The product name is: <strong>Nike Tiempo Legend 10 Elite</strong>
```

---

**User:** What is the size?
**Response:**

```html
The size is: <strong>8.5</strong>
```

---

**User:** Show me the product image.
**Response:**

```html
<Image src="https://static.nike.com/a/images/t_default/53496442-075a-4e82-9fb4-8759b8e7285d/LEGEND+10+ELITE+AG-PRO.png" alt="Product image" layout="fill" objectFit="contain" />
```

---

**User:** Give me the full order summary.
**Response:**

```html
<div>
  <p><strong>Status:</strong> Pending</p>
  <p><strong>Tracking ID:</strong> ASM90329313</p>
  <p><strong>Expected Delivery:</strong> 2025-05-03</p>
  <p><strong>Shipping Address:</strong> village lakra sialkot, Pakistan</p>
  <p><strong>Product:</strong> Nike Tiempo Legend 10 Elite</p>
  <p><strong>Quantity:</strong> 2</p>
  <p><strong>Color:</strong> Metallic Silver/Volt/Black</p>
  <p><strong>Size:</strong> 8.5</p>
  <Image src="https://static.nike.com/a/images/t_default/53496442-075a-4e82-9fb4-8759b8e7285d/LEGEND+10+ELITE+AG-PRO.png" alt="Product image" layout="fill" objectFit="contain" />
</div>
```

---

✅ Be polite, clear, and helpful.
🛑 Never return unrelated or unstructured text.
📦 Always provide **only** the info the user asks for, formatted in clean HTML.
"""

        return system_prompt
    except Exception as e:
        return {"error": f"An unexpected error occurred while fetching the order."}


OrderTracking = Agent(
    name="order_assistant_agent",
    instructions="""
        """,
    model=model,
)

In [5]:
def dynamic_instruction(context, agent): 
    result = {
            "message": "Order(s) retrieved successfully.",
            "items": [
                {
                    "shipping_address": "village lakra sialkot,pakistan",
                    "tracking_id": "ASM90329313",
                    "status": "pending",
                    "expected_delivery_date": "2025-05-03",
                    "created_at": "2025-04-26",
                    "items": [
                        {
                            "product_name": "Nike Tiempo Legend 10 Elite",
                            "price": 240.0,
                            "quantity": 2,
                            "color": "Metallic Silver/Volt/Black",
                            "size": 8.5,
                            "image": "https://static.nike.com/a/images/t_default/53496442-075a-4e82-9fb4-8759b8e7285d/LEGEND+10+ELITE+AG-PRO.png",
                        }
                    ],
                }
            ],
        }
    order = result["items"][0]
    product = order["items"][0]

    tracking_id = order["tracking_id"]
    status = order["status"]
    expected_delivery = order["expected_delivery_date"]
    shipping_address = order["shipping_address"]

    product_name = product["product_name"]
    quantity = product["quantity"]
    color = product["color"]
    size = product["size"]
    image_url = product["image"] 
    full_summary_html = f"""
    <div>
    <p><strong>Status:</strong> {status}</p>
    <p><strong>Tracking ID:</strong> {tracking_id}</p>
    <p><strong>Expected Delivery:</strong> {expected_delivery}</p>
    <p><strong>Shipping Address:</strong> {shipping_address}</p>
    <p><strong>Product:</strong> {product_name}</p>
    <p><strong>Quantity:</strong> {quantity}</p>
    <p><strong>Color:</strong> {color}</p>
    <p><strong>Size:</strong> {size}</p>
    <Image src="{image_url}" alt="Product image" layout="fill" objectFit="contain" />
    </div>
    """

    system_prompt = f"""
    You are a smart and helpful HTML-based order tracking assistant for ArmanStore.

    ✅ Your job is to:
    - Return **only** what the user asks for (e.g., color, image, size).
    - Format the response using valid **HTML**, so it renders correctly in a Next.js frontend.
    - Never fabricate details or provide unrelated information.
    - Always use the actual data provided below.

    ---

    ### 📦 Order Snapshot (for you to use):

    - Product: {product_name}  
    - Quantity: {quantity}  
    - Size: {size}  
    - Color: {color}  
    - Status: {status}  
    - Tracking ID: {tracking_id}  
    - Delivery Date: {expected_delivery}  
    - Shipping Address: {shipping_address}  
    - Image URL: {image_url}

    ---

    ### 🔁 Examples of HTML Responses

    **User:** What is the product name?  
    **Response:**
    ```html
    The product name is: <strong>{product_name}</strong>
    ````

    ---

    **User:** What is the size?
    **Response:**

    ```html
    The size is: <strong>{size}</strong>
    ```

    ---

    **User:** Show me the product image.
    **Response:**

    ```html
    <Image src="{image_url}" alt="Product image" layout="fill" objectFit="contain" />
    ```

    ---

    **User:** Give me the full order summary.
    **Response:**
    {full_summary_html}

    ✅ Keep responses clean, short, and 100% accurate.
    """
    return system_prompt

OrderTracking = Agent(
    name="order_assistant_agent",
    instructions=dynamic_instruction,
    model=model,
)

In [8]:

resp = await Runner.run(OrderTracking, "I want to track my order my tracking id is ASM90329313 what product name the total price and give me the color",max_turns=2)
print(resp.final_output)

```html
<p><strong>Product:</strong> Nike Tiempo Legend 10 Elite</p>
<p><strong>Color:</strong> Metallic Silver/Volt/Black</p>
```


In [3]:
from pydantic import BaseModel
from typing import Literal,Union
from agents import Agent 
class CategoryOnly(BaseModel):
    category: Literal[
        "OrderTracking", 
        "CustomerComplaint",
        "ComplaintStatus",
        "KnowledgeBaseQA", 
        "ProductRecommendation",
    ]
class DirectReply(BaseModel):
    message: str

CategorizerType = Union[CategoryOnly, DirectReply]

categorizer = Agent(
    name="categorizer",
    instructions = (
    "You are an intelligent orchestration agent responsible for routing user queries "
    "to the most appropriate specialized sub-agent based on the user's intent.\n\n"

    "Your job is to either:\n"
    "- Return a category in JSON like { \"category\": \"SubAgentName\" }, OR\n"
    "- Reply directly using { \"message\": \"<response>\" } if the query is off-topic or just a greeting.\n\n"

    "### Categories:\n"
    "1. **OrderTracking** – For all queries related to a user's orders, including:\n"
    "   - Checking the status or shipping progress using a tracking ID.\n"
    "   - Viewing a user's full order history using their email address.\n\n"

    "2. **CustomerComplaint** – When the user wants to raise a concern, report an issue, or file a complaint about a shoe or delivery. Typically requires a `tracking_id`.\n\n"

    "3. **ComplaintStatus** – When the user wants to check the status or resolution progress of a previously submitted complaint. Requires a `complaint_id`.\n\n"

    "4. **KnowledgeBaseQA** – For general questions about store policies, payments, returns, warranties, product care, or shoe sizing. This is powered by a Retrieval-Augmented Generation (RAG) system.\n\n"

    "5. **ProductRecommendation** – When the user requests personalized shoe suggestions based on filters like size, color, price, product type, or discounts.\n\n"

    "### Classification Examples:\n"
    '- "Can you track my shipment?" → { "category": "OrderTracking" }\n'
    '- "What shoes did I buy last month?" → { "category": "OrderTracking" }\n'
    '- "Where is my order TRK00123?" → { "category": "OrderTracking" }\n'
    '- "I received the wrong size" → { "category": "CustomerComplaint" }\n'
    '- "What’s the update on my complaint from last week?" → { "category": "ComplaintStatus" }\n'
    '- "How do I return a damaged product?" → { "category": "KnowledgeBaseQA" }\n'
    '- "Suggest white sneakers in size 10 under $100" → { "category": "ProductRecommendation" }\n\n'

    "### Special Cases:\n"
    "- If the user's question is outside your domain (e.g., news, politics, programming, ChatGPT), respond with:\n"
    '  { "message": "I’m ArmanStore’s assistant. I can help with questions about our shoes, orders, and complaints only." }\n\n'

    "- If the user greets you (e.g., 'Hi', 'Hello', 'How are you?'), respond with:\n"
    '  { "message": "Hi there! I’m here to help you with anything related to ArmanStore’s shoes. What would you like to know?" }\n\n'

    "**Never guess categories. Only return what's accurate. Do not hallucinate or make assumptions.**"
), 
    model=model,
    output_type=CategorizerType,
)


In [17]:
from agents import function_tool,Agent  

@function_tool(strict_mode=False)
def recommend_product(
    size: float,
    gender:str=None,
    color: str = None,
    discount: float = None,
    price: float = None,
    for_use: str = None, 
):
    """
    Recommends a product based on customer preferences.

    Args:
        size (float): Customer's preferred shoe size in US or EU sizing (e.g., 9.5 or 42.5).
        gender (str, optional): One of ["MALE", "FEMALE", "BOTH"]
        color (str, optional): Preferred color of the shoe (e.g., "Black", "White").
        discount (float, optional): Minimum discount required, as a percentage (e.g., 10.0 for 10%).
        price (float, optional): Maximum price the user is willing to pay (e.g., 100.0).
        for_use (str, optional): Intended use of the product, e.g., "Running", "Basketball", etc. 

    Returns:
        str: A message recommending the most suitable product or a message indicating no matches found.
    """
    
    
     
    my_values= (size, gender,color, discount, price, for_use)
    for i in my_values:
        print(i)
    # return my_values 
        

ProductRecommendation = Agent(
    name="user_recommendation_agent",
      instructions=(
        "You are a smart shoe recommendation assistant at **ArmanStore**.\n\n"
        "Your task is to help customers find the most suitable shoe products based on their preferences.\n\n"
        
        "**Required fields:**\n"
        "- `size` (US or EU shoe size)\n\n"
        
        "**Optional filters:**\n"
        "- `gender`: Must be one of `MALE`, `FEMALE`, `BOTH` (case-insensitive)\n"
        "- `color` (e.g., black,white,university red)\n"
        "- `discount` (minimum discount the user prefers)\n\n"
        "- `price` (maximum price the user is willing to pay)\n"
        "- `for_use` (e.g., Training, Running, Casual)\n"   
        
        "**For Use Matching Rule:**\n"
        "If the user provides a `for_use` preference, match it exactly against the following values:\n"
        "`Basketball`, `Casual`, `Casual Play`, `Casual Wear`, `Casual, Everyday`, `Casual, Running`, `Competition`, `Court Sports`, `Cross Training`, `Fitness`, `Hiking`, `Indoor Soccer`, `Indoor Sports`, `Indoor and Outdoor`, `Lifestyle`, `Marathon Running`, `Multi-sport`, `Play/Indoor Sports`, `Playground, Running`,`Road Running`, `Running`, `Soccer`, `Sports Training`, `Tennis`, `Trail Running`, `Training`, `Walking`,`Walking, Everyday`, `Water Activities`, `Weight Training`, `Weightlifting`, `Wrestling`\n"
        
        "**Size Interpretation Rule:**\n"
        "- The system supports US sizes from 1 to 15 and EU sizes from 33 to 47.\n"
        "- If the user says things like 'US 10', 'size 10', or just a number between 1–15, treat it as `size`.\n"
        "- If the user says 'EU 44', 'size 42', or mentions a number between 32–49, treat it as `size`.\n"
        "- If the size is outside supported ranges, ignore it or ask the user to provide a valid size.\n\n"
        
        "**Behavior Rules:**\n"
        "- If size is missing or invalid, politely ask the user to provide a valid shoe size.\n"
        "- If gender is provided but invalid, inform the user of allowed values.\n"
        "- Always call the `recommend_product` tool with all available valid inputs.\n"
        "- If no matching products found, inform the user.\n"
        "Note:"
        " If the message is vague like: 'I want to buy shoes like my friend/teacher/colleague', and you can't find size, reply:\n"
        "'To help you find the exact product, please provide the size of your shoes — or upload a clear side-view photo of your [friend’s/teacher’s/colleague’s] shoes.'\n\n"

    ),
    model=model,
    tools=[recommend_product], 
    tool_use_behavior='stop_on_first_tool'
)


In [15]:

resp = await Runner.run(ProductRecommendation, "I want to buy shoes like my friend",max_turns=2)
print(resp.final_output)

To help you find the exact product, please provide the size of your shoes — or upload a clear side-view photo of your friend’s shoes.



In [ ]:

resp = await Runner.run(ProductRecommendation, "I want to buy red shoes like my friend whose size is 12",max_turns=1)
print(resp.final_output)

12.0
None
red
None
None
None
None


In [19]:

resp = await Runner.run(ProductRecommendation, "I want to buy red shoes like my friend whose size is 12 range is 300 and discount is 10 for running",max_turns=1)
print(resp.final_output)

12.0
None
red
10.0
300.0
Running
None


In [25]:

resp = await Runner.run(ProductRecommendation, "I want to buy  red shoes like my friend whose size is 12 range is 300 and discount is 10 for running for kids",max_turns=1)
print(resp.final_output)

I can help with that! However, I need a valid size and gender.
For size, I support US sizes from 1 to 15 and EU sizes from 33 to 47.
For gender, I support MALE, FEMALE, and BOTH.
Could you please provide these details?



In [ ]:

resp = await Runner.run(ProductRecommendation, "I want to buy  red shoes like  price range is 300 and discount is 10 for running for myself",max_turns=1)
print(resp.final_output)

What is your shoe size?


In [ ]:
resp = await Runner.run(categorizer, "hi", max_turns=2)
final_resp = resp.final_output   

if isinstance(final_resp, CategoryOnly):
    print("Category:", final_resp.category)
elif isinstance(final_resp, DirectReply):
    print("Message:", final_resp.message)
else:
    print("Unknown response type")


Message: Hi there! I’m here to help you with anything related to ArmanStore’s shoes. What would you like to know?


In [16]:
resp = await Runner.run(categorizer, "what is the next step we buy last month shoes",max_turns=1)
print(resp.final_output)

category='OrderTracking'


In [5]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper,enable_verbose_stdout_logging


enable_verbose_stdout_logging()

@function_tool 
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    
    resp = f"Prime Mi nister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021",max_turns=2)
print(resp.final_output)

Creating trace Agent workflow with id trace_7ff7fc0b68a24c528aa7803d52f16b2d
Creating trace Agent workflow with id trace_7ff7fc0b68a24c528aa7803d52f16b2d
Setting current trace: trace_7ff7fc0b68a24c528aa7803d52f16b2d
Setting current trace: trace_7ff7fc0b68a24c528aa7803d52f16b2d
Creating span <agents.tracing.span_data.AgentSpanData object at 0x00000290F14E9A90> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x00000290F14E9A90> with id None
Running agent tool agent (turn 1)
Running agent tool agent (turn 1)
Creating span <agents.tracing.span_data.GenerationSpanData object at 0x00000290EE1B9250> with id None
Creating span <agents.tracing.span_data.GenerationSpanData object at 0x00000290EE1B9250> with id None
[
  {
    "content": "You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.",
    "role": "system"
  },
  {
    "role": "user",
    "content": "who is prime Min

In [4]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    pass
    # return f"⚠️ Tool error: {str(exception)}"

@function_tool 
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021",max_turns=1)
print(resp.final_output)

ModuleNotFoundError: No module named 'agents'

In [19]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 60 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = Runner.run_sync(agent, "who is prime Minster of Pakistan in 2021 and what is the weather condition of lahore",max_turns=3)
print(resp.final_output)

 Prime Minster of Pakistan in 2021 is imran khan 
l Weather in lahore are very beautiful and charming l
Pakistan ke wazir-e-azam 2021 mein Imran Khan hain, aur Lahore ka mausam bohat khushgawar hai.



In [20]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)
enable_verbose_stdout_logging()
@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 60 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = Runner.run_sync(agent, "who is prime Minster of Pakistan in 2021 and what is the weather condition of lahore",max_turns=2)
print(resp.final_output)

Creating trace Agent workflow with id trace_a35e79752b9545c4b982e748d51c1a14
Setting current trace: trace_a35e79752b9545c4b982e748d51c1a14
Creating span <agents.tracing.span_data.AgentSpanData object at 0x000001D0915BEB20> with id None
Running agent tool agent (turn 1)
Creating span <agents.tracing.span_data.GenerationSpanData object at 0x000001D09161C3B0> with id None
[
  {
    "content": "You are an helpful assistant response in natural language like in urdu and clear brief not more than 60 words if user query related to tool call. Call the tool call.",
    "role": "system"
  },
  {
    "role": "user",
    "content": "who is prime Minster of Pakistan in 2021 and what is the weather condition of lahore"
  }
]
Tools:
[
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "",
      "parameters": {
        "properties": {
          "city": {
            "title": "City",
            "type": "string"
          }
        },
        "required": [
  

In [38]:
from agents import Agent, ModelSettings, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)
# enable_verbose_stdout_logging()
@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31m Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 60 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool',
    # model_settings=ModelSettings(parallel_tool_calls=True),
    model_settings=ModelSettings(parallel_tool_calls=None)
)

resp = Runner.run_sync(agent, "who is prime Minster of Pakistan in 2021 and what is the weather condition of lahore",max_turns=2)
print('final',resp.final_output)
outputs = [item.output for item in resp.new_items if hasattr(item, "output")]
final_response = "\n".join(outputs)
print("✅ Combined Response:\n", final_response)

Creating trace Agent workflow with id trace_a6e88d86de4b4bb9a9e31116ccceda48
Setting current trace: trace_a6e88d86de4b4bb9a9e31116ccceda48
Creating span <agents.tracing.span_data.AgentSpanData object at 0x000001D092032300> with id None
Running agent tool agent (turn 1)
Creating span <agents.tracing.span_data.GenerationSpanData object at 0x000001D09206BFB0> with id None
[
  {
    "content": "You are an helpful assistant response in natural language like in urdu and clear brief not more than 60 words if user query related to tool call. Call the tool call.",
    "role": "system"
  },
  {
    "role": "user",
    "content": "who is prime Minster of Pakistan in 2021 and what is the weather condition of lahore"
  }
]
Tools:
[
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "",
      "parameters": {
        "properties": {
          "city": {
            "title": "City",
            "type": "string"
          }
        },
        "required": [
  

In [3]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = Runner.run_sync(agent, "who is prime Minster of Pakistan in 2021",max_turns=1)
print(resp.final_output)

OPENAI_API_KEY is not set, skipping trace export


 Prime Minster of Pakistan in 2021 is imran khan 


OPENAI_API_KEY is not set, skipping trace export


MaxTurnsExceeded: Max turns (1) exceeded

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is no

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is no

In [4]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = Runner.run_streamed(agent, "who is prime Minster of Pakistan in 2021")
async for rep in resp.stream_events():
    print(rep)
print(resp.final_output)

AgentUpdatedStreamEvent(new_agent=Agent(name='tool agent', instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.', prompt=None, handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001D08A52D550>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[FunctionTool(name='get_weather', description='', params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function

In [5]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
async def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
async def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of Pakistan in 2021")
print(resp.final_output)

 Prime Minster of Pakistan in 2021 is imran khan 
پاکستان کے وزیراعظم 2021 میں عمران خان تھے۔



In [148]:
from doctest import debug
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, Usage, function_tool,AgentHooks
class TurnCounter(AgentHooks):
    def __init__(self):
        self.turns = 0
        self.tool_calls = 0

    async def on_start(self, context: RunContextWrapper, agent: Agent):
        self.turns += 1
        print(f"\n🔄 Turn {self.turns} started (AI thinking)")

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool):
        self.tool_calls += 1
        print(f"🛠️ Tool `{tool.name}` called")
class CustomAgentHooks(AgentHooks):
    def __init__(self, display_name: str):
        self.event_counter = 0
        self.display_name = display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started")
    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started tool {tool.name}"
        )

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],hooks=TurnCounter(),
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of Pakistan in 2021")
print(resp.final_output)


🔄 Turn 1 started (AI thinking)
🛠️ Tool `find_primenister` called
 Prime Minster of Pakistan in 2021 is imran khan 
پاکستان کے وزیراعظم 2021 میں عمران خان تھے۔



In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool
def find_primenister(country:str,year:int=2025)->str:
    """ 
    This function logic find the prime minster of the country
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

 Prime Minster of pakistan in 20000 is imran khan 
Pakistan ke Prime Minister 20000 mein Imran Khan hain.



In [6]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool
def find_primenister(country:str,year:int=2025)->str:
    """ 
    This function logic find the prime minster of the country
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

 Prime Minster of pakistan in 20000 is imran khan 
Pakistan ke prime minister 20000 mein Imran Khan hain.



In [7]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2100:
        
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

UserError: Error running tool find_primenister: Year is too far in the future. Please enter a valid year.

In [8]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool()
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2100:
        
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

An error occurred while running the tool. Please try again. Error: Year is too far in the future. Please enter a valid year.


In [100]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    # tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

UserError: Error running tool find_primenister: Year is too far in the future. Please enter a valid year.

In [9]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

⚠️ Tool error: Year is too far in the future. Please enter a valid year.


In [101]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

⚠️ Tool error: Year is too far in the future. Please enter a valid year.


In [10]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    # tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

The year is too far in the future. Please enter a valid year.



In [104]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    pass
    # return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=my_custom_error_handler)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    # tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

I am unable to find the prime minister of Pakistan in 2021.



In [108]:
from agents import Agent, Runner, set_tracing_disabled, function_tool
 
import asyncio

set_tracing_disabled(disabled=False)

@function_tool(failure_error_function=None)
def extract_fun(a: str, b: int):
    """Add two numbers.

    Args:
        a: First argument.
        b: Second argument.
    """
    print(f"{a},{b}")
    print(f"{a},{b},{a+b}")
    return {a + b}

agent = Agent(
    name="test-agent",
    instructions="""
    You must call the `extract_fun` tool using exactly these values: a = 'arman', b = 'twenty four'.
    Do not ask the user for clarification. Do not rephrase. Just call the tool.
    """,
    tools=[extract_fun],
    model=model,
    
)

async def main():
    try:
        resp = await Runner.run(agent, "Please follow instructions strictly.")
        print(resp.final_output)
    except Exception as e:
        print(f"❌ Caught Exception: {type(e).__name__} - {e}")

asyncio.run(main())


arman,24
❌ Caught Exception: UserError - Error running tool extract_fun: can only concatenate str (not "int") to str


In [109]:
from agents import Agent, Runner, set_tracing_disabled, function_tool
import asyncio

set_tracing_disabled(disabled=False)

@function_tool(failure_error_function=None)
def extract_fun(a: str, b: int):
    """Add two numbers.

    Args:
        a: First argument.
        b: Second argument.
    """
    print(f"{a},{b},{a + b}")

agent = Agent(
    name="test-agent",
    instructions="""
    You must call the `extract_fun` tool. Do NOT correct your JSON or ask the user. 
    Provide your tool call in a single JSON object (no code fences).
    """,
    tools=[extract_fun],
    model=model, 
)

async def main():
    try:
        # Here we force the model to emit malformed JSON:
        resp = await Runner.run(agent,
            "Call extract_fun now with this exact text: { a: arman, b: }"
        )
        print("Final Output:", resp.final_output)
    except Exception as e:
        # This should be ModelBehaviorError
        print(f"❌ Caught Exception: {type(e).__name__} - {e}")

asyncio.run(main())


Final Output: I'm sorry, I cannot fulfill your request. The value for 'b' is missing.



In [16]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
 
from agents import function_tool

@function_tool()
def extract_fun(a:str, b:int):
    """Add two numbers.
   
    Args:
        a : First argument   .
        b : Second argument .
    """
    print(f"{a},{b},{a+b}")
    # return a + b


agent = Agent(
    name="test-agent",
    instructions="Always call the extract_fun tool.",
    tools=[extract_fun],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

try:
    resp = await Runner.run(agent, "  first value is name and its value is arman and second value is age and its value is 24")

    print(resp.final_output)
except Exception as e:
    print("❌ Tool failed:", type(e).__name__, "-", e)    

An error occurred while running the tool. Please try again. Error: can only concatenate str (not "int") to str


In [17]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
 
from agents import function_tool

@function_tool(failure_error_function=None)
def extract_fun(a:str, b:int):
    """Add two numbers.
   
    Args:
        a : First argument   .
        b : Second argument .
    """
    print(f"{a},{b},{a+b}")
    # return a + b


agent = Agent(
    name="test-agent",
    instructions="Always call the extract_fun tool.",
    tools=[extract_fun],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

try:
    resp = await Runner.run(agent, "  first value is name and its value is arman and second value is age and its value is 24")

    print(resp.final_output)
except Exception as e:
    print("❌ Tool failed:", type(e).__name__, "-", e)    

❌ Tool failed: UserError - Error running tool extract_fun: can only concatenate str (not "int") to str


In [11]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
 
@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)
try:
    resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
    print(resp.final_output)
except Exception as e:
    print("❌ Tool failed:", type(e).__name__, "-", e)    

❌ Tool failed: UserError - Error running tool find_primenister: Year is too far in the future. Please enter a valid year.


In [13]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
 
@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        return 3+'4'
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)
try:
    resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
    print(resp.final_output)
except Exception as e:
    print("❌ Tool failed:", type(e).__name__, "-", e)    

❌ Tool failed: UserError - Error running tool find_primenister: unsupported operand type(s) for +: 'int' and 'str'


In [53]:
from agents import Agent, Runner, set_tracing_disabled,function_tool,RunContextWrapper

set_tracing_disabled(disabled=False)

 
def my_custom_error_handler( ctx: RunContextWrapper,exception: Exception,) -> str:
    pass
    # return f"⚠️ Tool error: {str(exception)}"

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    if year > 2020: 
        raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 2021")
print(resp.final_output)

UserError: Error running tool find_primenister: Year is too far in the future. Please enter a valid year.

In [38]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

 

@function_tool(failure_error_function=None)
def find_primenister(country: str, year: int = 2025) -> str:
    """ 
    This function logic finds the prime minister of the country.
    
    Args:
        country (str): Name of the country
        year (int): Year of the country
    """
    # if year > 2100:
    #     raise ValueError("Year is too far in the future. Please enter a valid year.")
    
    resp = f"Prime Minister of {country} in {year} is Imran Khan"
    print(f"\033[31m {resp} \033[0m")
    return resp

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language ,clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
       find_primenister
    ],
    model=model,
    tool_use_behavior='stop_on_first_tool'
)

resp = await Runner.run(agent, "who is prime Minster of pakistan in 20000")
print(resp.final_output)

 Prime Minister of pakistan in 20000 is Imran Khan 
Prime Minister of pakistan in 20000 is Imran Khan


In [ ]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool(name_override='primenister')
def find_primenister(country:str,year:str=2025)->str:
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

resp = await Runner.run(agent, "who is prime Minster of india in 2021")
print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         # Output: greet
    print(tool.description) 

 Prime Minster of india in 2021 is imran khan 
بھارت کے وزیر اعظم 2021 میں عمران خان ہیں۔



In [ ]:
 # Output: Greet the user by name.

get_weather

primenister



In [19]:
from agents import Agent, Runner, set_tracing_disabled,function_tool

set_tracing_disabled(disabled=False)

@function_tool
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''Find the priminter of the country'''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         # Output: greet
    print(tool.description) 

get_weather

primenister
This function is used to find the prime Minster of country


In [13]:
from agents import Agent, Runner, set_tracing_disabled,function_tool
import json
set_tracing_disabled(disabled=False)

@function_tool()
def get_weather(city:str)->str:
     resp =  f'Weather in {city} are very beautiful and charming'
     print(f'\033[31ml Weather in {city} are very beautiful and charming \033[0ml')
     return resp

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''Find the priminter of the country'''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[
        get_weather,find_primenister
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         # Output: greet
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

get_weather

{
  "properties": {
    "city": {
      "title": "City",
      "type": "string"
    }
  },
  "required": [
    "city"
  ],
  "title": "get_weather_args",
  "type": "object",
  "additionalProperties": false
}
primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [29]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country',use_docstring_info=False)
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_primenister

{
  "properties": {
    "country": {
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "find_primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [113]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country' )
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from pydantic import BaseModel
from dataclasses import dataclass
from typing_extensions import TypedDict
from pydantic import BaseModel,Field

set_tracing_disabled(disabled=False)
@dataclass
class custom_class:
    name: str = Field(description="name of the custom_class")
    
   

 
@function_tool
def find_president(country:str,cus:custom_class,year:str=2025,)->str:
    '''
    Find the president of the country
        
    Args:
        country (int): Name of the country
        year (int): Year of the country
        cus (custom_class): custom object 
        '''
    resp =  f"prisident of {country} in {year} is imran khan"
    print(f"\033[31m prisident of {country} in {year} is imran khan\033[0m ")
    return resp 
 
agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[ find_president
    ],
    model=model,
) 
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_president
Find the president of the country
{
  "$defs": {
    "custom_class": {
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "cus": {
      "description": "custom object\nname (str): The name of the person to include.",
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": 2025,
      "description": "Y

In [18]:
from typing import Literal

def get_status(code: Literal["success", "error"]) -> str:
    return f"Status: {code}"

# get_status("success")  
get_status(1)      


'Status: 1'

In [130]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from pydantic import BaseModel
from dataclasses import dataclass
from typing_extensions import TypedDict
from pydantic import BaseModel,Field

set_tracing_disabled(disabled=False)

class custom_class(BaseModel):
    name: str = Field(description="name of the custom_class")
    
   

 
@function_tool
def find_president(country:str,cus:custom_class,year:str=2025,)->str:
    '''
    Find the president of the country
        
    Args:
        country (int): Name of the country
        year (int): Year of the country
        cus (custom_class): custom object 
        '''
    resp =  f"prisident of {country} in {year} is imran khan"
    print(f"\033[31m prisident of {country} in {year} is imran khan\033[0m ")
    return resp 
 
agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[ find_president
    ],
    model=model,
) 
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_president
Find the president of the country
{
  "$defs": {
    "custom_class": {
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "cus": {
      "description": "custom object",
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
    

In [134]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from pydantic import BaseModel
from dataclasses import dataclass
from typing_extensions import TypedDict
from pydantic import BaseModel,Field

set_tracing_disabled(disabled=False)

class custom_class(BaseModel):
    name: str = Field(description="name of the custom_class")
    
   

 
@function_tool
def find_president(country:str,cus:custom_class,year:str=2025,)->str:
    '''
    Find the president of the country
        
    Args:
        country (int): Name of the country
        year (int): Year of the country
        cus (custom_class): custom object 
        '''
    resp =  f"prisident of {country} in {year} is imran khan"
    print(f"\033[31m prisident of {country} in {year} is imran khan\033[0m ")
    return resp 
 
agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[ find_president
    ],
    model=model,
) 
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

find_president
Find the president of the country
{
  "$defs": {
    "custom_class": {
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "cus": {
      "description": "custom object",
      "properties": {
        "name": {
          "description": "name of the custom_class",
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "custom_class",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
    

In [ ]:
from typing_extensions import TypedDict
from agents import function_tool, Agent
import json
 
class Custom(TypedDict):
    name: str

@function_tool
def find_president(
    country: str,
    custom: Custom,
    year: str = "2025"
) -> str:
    """
    Find the president of the country.

    Args:
        country (str): Name of the country
        custom (Custom): Extra info
        year (str): Year (default 2025)
    """
    return f"President of {country} in {year} with {custom['name']}"

# 2) Register and inspect
agent = Agent(name="tool-agent", tools=[find_president], model=model)
for tool in agent.tools:
    print(json.dumps(tool.params_json_schema, indent=2))


{
  "$defs": {
    "Custom": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "Custom",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "custom": {
      "description": "Extra info",
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "Custom",
      "type": "object",
      "additionalProperties": false
    },
    "year": {
      "default": "2025",
      "description": "Year (default 2025)",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "custom",
    "year"
  ],
  "title": "find_president_args",
  "type": "object",
  "additionalProperties": 

In [35]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: ''
        year: str
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "''",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "str",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [30]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
set_tracing_disabled(disabled=False)
 

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

In [18]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from typing_extensions import  TypedDict
set_tracing_disabled(disabled=False)
class Location(TypedDict):
    lat: float
    long: float

@function_tool()
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        
    Args:
        country: Name of the country
        year: Year of the country
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister,fetch_weather,
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}
fetch_weather
Fetch the weather for a given location.
{
  "$defs": {
    "Location": {
      "properties": {
        "lat": {
          "title": "Lat",
          "type": "number"
        },
        "long": {
          "title": "Long",
          "type": "number"
        }
      },
      "required": [
        "lat",
        "long"
      ],
      "title": "Location",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "location": {
      "description": "The location to fetch the 

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from typing_extensions import  TypedDict
set_tracing_disabled(disabled=False)
class Location(TypedDict):
    lat: float
    long: float

@function_tool()
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location (str): The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

@function_tool(name_override='primenister',description_override='This function is used to find the prime Minster of country')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        this function finds the primenister of the country
    args:
        country (str): Name of the country
        year (str): Year of the country
    Returns:
        str : country prime Minster    
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister, 
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
This function is used to find the prime Minster of country
{
  "properties": {
    "country": {
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [26]:
from agents import Agent, Runner, set_tracing_disabled
from agents.tool import function_tool
import json
from typing_extensions import  TypedDict
set_tracing_disabled(disabled=False)
class Location(TypedDict):
    lat: float
    long: float

@function_tool()
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location (str): The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

@function_tool(name_override='primenister')
def find_primenister(country:str,year:str=2025)->str:
    '''
    Find the priminter of the country
        this function finds the primenister of the country
        
    args:
        country (str): Name of the country
        year (str): Year of the country
    Returns:
        str : country prime Minster    
        '''
    resp =  f"Prime Minster of {country} in {year} is imran khan"
    print(f"\033[31m Prime Minster of {country} in {year} is imran khan\033[0m ")
    return resp 

agent = Agent(
    name="tool agent",
    instructions='You are an helpful assistant response in natural language like in urdu and clear brief not more than 20 words if user query related to tool call. Call the tool call.',
    tools=[find_primenister, 
    ],
    model=model,
)

# resp = await Runner.run(agent, "who is prime Minster of india in 2021")
# print(resp.final_output)
for tool in agent.tools:
    print(tool.name)         
    print(tool.description) 
    print(json.dumps(tool.params_json_schema, indent=2))

primenister
Find the priminter of the country
    this function finds the primenister of the country
{
  "properties": {
    "country": {
      "description": "Name of the country",
      "title": "Country",
      "type": "string"
    },
    "year": {
      "default": 2025,
      "description": "Year of the country",
      "title": "Year",
      "type": "string"
    }
  },
  "required": [
    "country",
    "year"
  ],
  "title": "primenister_args",
  "type": "object",
  "additionalProperties": false
}


In [4]:
import json
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool, Agent
 
def do_some_work(data: str) -> str:
    return "done" 
class FunctionArgs(BaseModel):
    username: str
    age: int 
async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")
 
custom_tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)
 
agent = Agent(
    name="CustomToolAgent",
    tools=[custom_tool],
)
 
for tool in agent.tools:
    print("🛠 Tool Name:", tool.name)
    print("📝 Description:", tool.description)
    print("📦 Input Schema:")
    print(json.dumps(tool.params_json_schema, indent=2))


🛠 Tool Name: process_user
📝 Description: Processes extracted user data
📦 Input Schema:
{
  "properties": {
    "username": {
      "title": "Username",
      "type": "string"
    },
    "age": {
      "title": "Age",
      "type": "integer"
    }
  },
  "required": [
    "username",
    "age"
  ],
  "title": "FunctionArgs",
  "type": "object"
}


In [ ]:
import json
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool, Agent
 
def do_some_work(data: str) -> str:
    return "done" 
class FunctionArgs(BaseModel):
    username: str
    age: int 
    
class newFunctionArgs(BaseModel):
    username: str
    age: int 
async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")
 
custom_tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)
 
agent = Agent(
    name="CustomToolAgent",
    tools=[custom_tool],
)
 
for tool in agent.tools:
    print("🛠 Tool Name:", tool.name)
    print("📝 Description:", tool.description)
    print("📦 Input Schema:")
    print(json.dumps(tool.params_json_schema, indent=2))


ModuleNotFoundError: No module named 'griffe.loader'